# 🏥 TanıAI Cloud Kurulumu

Google Colab/Kaggle ortamında Whisper + ML + LLM entegre sistemi

## 🎯 Özellikler
- **Whisper Medium Model**: Ses dosyalarını Türkçe metne çevirir
- **ML Triage Model**: Scikit-learn tabanlı klinik öneri sistemi
- **LLM Entegrasyonu**: Ollama + küçük modeller (0.5B-2B parametre)
- **Hibrit Sistem**: ML + LLM sonuçlarını birleştirerek daha doğru öneriler


## 📦 1. Gerekli Paketleri Kur


In [ ]:
# Gerekli paketleri kur
%pip install openai-whisper torch torchaudio scikit-learn pandas numpy fastapi uvicorn python-multipart requests aiohttp

print("✅ Tüm paketler kuruldu!")


## 🤖 2. Ollama Kurulumu (LLM için)


In [ ]:
# Ollama'yı kur
!curl -fsSL https://ollama.ai/install.sh | sh

print("✅ Ollama kuruldu!")


In [ ]:
# Ollama servisini başlat (arka planda)
import subprocess
import time

# Ollama servisini başlat
ollama_process = subprocess.Popen(['ollama', 'serve'], 
                                 stdout=subprocess.DEVNULL, 
                                 stderr=subprocess.DEVNULL)

# Servisin başlamasını bekle
time.sleep(5)

print("✅ Ollama servisi başlatıldı!")


## 📥 3. Küçük Model İndir (2B Parametre)


In [ ]:
# Küçük model indir (2B parametre)
!ollama pull llama2:2b

print("✅ Llama2:2b modeli indirildi!")


## 🧪 4. Ollama Testi


In [ ]:
# Ollama modelini test et
import requests
import json

def test_ollama():
    url = "http://localhost:11434/api/generate"
    
    payload = {
        "model": "llama2:2b",
        "prompt": "Merhaba, nasılsın?",
        "stream": False
    }
    
    try:
        response = requests.post(url, json=payload, timeout=30)
        if response.status_code == 200:
            result = response.json()
            print("✅ Ollama çalışıyor!")
            print(f"Yanıt: {result.get('response', '')[:100]}...")
            return True
        else:
            print(f"❌ Ollama hatası: {response.status_code}")
            return False
    except Exception as e:
        print(f"❌ Ollama bağlantı hatası: {e}")
        return False

test_ollama()


## 📁 5. Proje Dosyalarını Yükle


In [ ]:
# Proje dosyalarını yükle
from google.colab import files
import zipfile
import os

print("📁 Proje dosyalarını yükleyin (zip formatında)")
uploaded = files.upload()

# Zip dosyasını çıkar
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall()
        print(f"✅ {filename} çıkarıldı")
        break

print("✅ Proje dosyaları hazır!")


## 🧪 6. Sistem Testi


In [ ]:
# ML modelini test et
import sys
sys.path.append('/content/RANDEVU')

try:
    from ml_clinic.triage_model import get_triage_model
    
    model = get_triage_model()
    result = model.suggest("Baş ağrım var ve mide bulantısı yaşıyorum", top_k=3)
    
    print("✅ ML Model çalışıyor!")
    print(f"Öneriler: {result.get('suggestions', [])}")
    
except Exception as e:
    print(f"❌ ML Model hatası: {e}")


In [ ]:
# Whisper modelini test et
import whisper

try:
    # Medium model kullan (cloud için optimize)
    model = whisper.load_model("medium")
    print("✅ Whisper Medium modeli yüklendi!")
    print(f"Model boyutu: {model.dims}")
    
except Exception as e:
    print(f"❌ Whisper hatası: {e}")


## 🤖 7. Cloud LLM Testi


In [ ]:
# Cloud LLM'i test et
try:
    from cloud_llm_analyzer import CloudLLMAnalyzer, CloudLLMConfig
    
    config = CloudLLMConfig(
        provider="ollama",
        model="llama2:2b",
        base_url="http://localhost:11434"
    )
    
    analyzer = CloudLLMAnalyzer(config)
    
    # Test metni
    test_text = "Baş ağrım var ve mide bulantısı yaşıyorum"
    result = analyzer.analyze_with_cloud_llm(test_text)
    
    print("✅ Cloud LLM çalışıyor!")
    print(f"Test: {test_text}")
    print(f"Öneri: {result.get('clinic', 'Bilinmeyen')}")
    print(f"Güven: {result.get('confidence', 0):.2f}")
    print(f"Açıklama: {result.get('reasoning', '')}")
    
except Exception as e:
    print(f"❌ Cloud LLM hatası: {e}")


## 🔄 8. Entegre Sistem Testi


In [ ]:
# Entegre sistemi test et
try:
    from ml_clinic.integrated_triage import get_integrated_triage, TriageConfig
    from ml_clinic.llm_clinic_analyzer import LLMConfig
    
    # Config'leri ayarla
    llm_config = LLMConfig(
        provider="ollama",
        model="llama2:2b",
        base_url="http://localhost:11434"
    )
    
    triage_config = TriageConfig(
        use_ml=True,
        use_llm=True,
        ml_weight=0.6,
        llm_weight=0.4
    )
    
    # Entegre sistemi başlat
    integrated_triage = get_integrated_triage(triage_config, llm_config)
    
    # Test senaryoları
    test_cases = [
        "Baş ağrım var ve mide bulantısı yaşıyorum",
        "Çocuğumda ateş ve öksürük var",
        "Göğüs ağrısı ve nefes darlığı çekiyorum",
        "Diş ağrısı dayanılmaz halde"
    ]
    
    print("✅ Entegre sistem çalışıyor!")
    print("\n📝 Test Sonuçları:")
    
    for i, test_text in enumerate(test_cases, 1):
        result = integrated_triage.suggest(test_text, top_k=3)
        print(f"\n{i}. {test_text}")
        print(f"   Öneriler: {result.get('suggestions', [])}")
        print(f"   Yöntemler: {result.get('methods_used', [])}")
        
except Exception as e:
    print(f"❌ Entegre sistem hatası: {e}")
